In [12]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [18]:
import numpy as np
import pandas as pd
from sklearn import neighbors, svm
from sklearn.model_selection import train_test_split

# Introdução ao Aprendizado de Máquina

## Trabalhando com dados

### 1. Load do dataset

Comparar dataset de regressão e classificação, abrir os datasets usando pandas.

In [19]:
weather_nominal = pd.read_csv('./weather.nominal.csv')

weather_numeric = pd.read_csv('./weather.csv')

weather_nominal
weather_numeric

,outlook,temperature,humidity,windy,play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes
5,rainy,cool,normal,True,no
6,overcast,cool,normal,True,yes
7,sunny,mild,high,False,no
8,sunny,cool,normal,False,yes
9,rainy,mild,normal,False,yes


,outlook,temperature,humidity,windy,play
0,sunny,85,85,False,no
1,sunny,80,90,True,no
2,overcast,83,86,False,yes
3,rainy,70,96,False,yes
4,rainy,68,80,False,yes
5,rainy,65,70,True,no
6,overcast,64,65,True,yes
7,sunny,72,95,False,no
8,sunny,69,70,False,yes
9,rainy,75,80,False,yes


Mostrando dataset com `head` e `tail` (bom para datasets extensos).

In [20]:
weather_nominal.head()
weather_numeric.tail()

,outlook,temperature,humidity,windy,play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes


,outlook,temperature,humidity,windy,play
9,rainy,75,80,False,yes
10,sunny,75,70,True,yes
11,overcast,72,90,True,yes
12,overcast,81,75,False,yes
13,rainy,71,91,True,no


In [23]:
nominal_train, nominal_test = train_test_split(weather_nominal, test_size=0.3)

nominal_train
nominal_test

,outlook,temperature,humidity,windy,play
8,sunny,cool,normal,False,yes
0,sunny,hot,high,False,no
9,rainy,mild,normal,False,yes
2,overcast,hot,high,False,yes
13,rainy,mild,high,True,no
10,sunny,mild,normal,True,yes
5,rainy,cool,normal,True,no
11,overcast,mild,high,True,yes
3,rainy,mild,high,False,yes


,outlook,temperature,humidity,windy,play
4,rainy,cool,normal,False,yes
12,overcast,hot,normal,False,yes
6,overcast,cool,normal,True,yes
7,sunny,mild,high,False,no
1,sunny,hot,high,True,no


In [24]:
numeric_train, numeric_test = train_test_split(weather_numeric, test_size=0.3)

numeric_train
numeric_test

,outlook,temperature,humidity,windy,play
11,overcast,72,90,True,yes
7,sunny,72,95,False,no
9,rainy,75,80,False,yes
12,overcast,81,75,False,yes
4,rainy,68,80,False,yes
8,sunny,69,70,False,yes
5,rainy,65,70,True,no
10,sunny,75,70,True,yes
6,overcast,64,65,True,yes


,outlook,temperature,humidity,windy,play
0,sunny,85,85,False,no
1,sunny,80,90,True,no
3,rainy,70,96,False,yes
13,rainy,71,91,True,no
2,overcast,83,86,False,yes


## Modelos

Usando o KNN para classificar as instâncias categóricas do `Weather`.

In [21]:
clas = neighbors.KNeighborsClassifier(n_neighbors=3)


Usando o SVM para fazer a regressão das instâncias numéricas do `Weather`.

In [ ]:
regr = svm.LinearSVC()


## Treino

## Avaliação de resultados